## 날씨 및 인구데이터 처리
(2019 기준)
- 기상청 날씨 데이터
- KOSIS 인구 데이터

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
print("pandas : ", pd.__version__)
print("numpy : ", np.__version__)

pandas :  1.0.5
numpy :  1.19.1


In [2]:
weather = pd.read_csv("weather.csv", encoding='cp949', engine='python')
weather.head()

,지점,지점명,일시,기온(°C),강수량(mm),습도(%)
0,90,속초,2019-01-01 01:00,-1.9,NaN,18.0
1,90,속초,2019-01-01 02:00,-2.2,NaN,17.0
2,90,속초,2019-01-01 03:00,-2.5,NaN,18.0
3,90,속초,2019-01-01 04:00,-3.9,NaN,20.0
4,90,속초,2019-01-01 05:00,-4.9,NaN,22.0


In [3]:
weather.isna().sum()

지점              0
지점명             0
일시              0
기온(°C)        131
강수량(mm)    757996
습도(%)         953
dtype: int64

강수량의 경우 결측이 대부분이고 습도로 대신할 수 있을 것으로 판단되기 때문에 제외해준다

In [4]:
weather = weather.drop(['강수량(mm)'], axis=1)

In [5]:
weather[pd.isnull(weather[['기온(°C)', '습도(%)']]).any(axis=1)]

,지점,지점명,일시,기온(°C),습도(%)
58662,101,춘천,2019-09-19 00:00,NaN,87.0
86212,105,강릉,2019-11-14 08:00,NaN,18.0
86213,105,강릉,2019-11-14 09:00,NaN,16.0
86214,105,강릉,2019-11-14 10:00,NaN,15.0
86215,105,강릉,2019-11-14 11:00,NaN,17.0
...,...,...,...,...,...
793026,285,합천,2019-09-03 02:00,NaN,93.0
793031,285,합천,2019-09-03 07:00,NaN,93.0
793033,285,합천,2019-09-03 09:00,NaN,93.0
793034,285,합천,2019-09-03 10:00,NaN,91.0


시단위였음..

In [11]:
# 아무튼 계속해보자
na = weather[pd.isnull(weather[['기온(°C)', '습도(%)']]).any(axis=1)].index

연속된 시간 혹은 비슷한 시간대에 결측이 일어났으므로 선형으로 비례하는 방식으로 결측값을 보간한다(interpolation)

> ### Pandas - interpolate()
> DataFrame 값에 선형으로 비례하는 방식으로 결측값 보간(method='values')
>> method='time'으로 설정할 경우 시계열 날짜 index를 기준으로 결측값 보간이 가능하다

In [7]:
w1 = weather.interpolate(method='values')

In [17]:
w1.iloc[list(na)]

,지점,지점명,일시,기온(°C),습도(%)
58662,101,춘천,2019-09-19 00:00,18.300000,87.0
86212,105,강릉,2019-11-14 08:00,2.920000,18.0
86213,105,강릉,2019-11-14 09:00,3.740000,16.0
86214,105,강릉,2019-11-14 10:00,4.560000,15.0
86215,105,강릉,2019-11-14 11:00,5.380000,17.0
...,...,...,...,...,...
793026,285,합천,2019-09-03 02:00,19.766667,93.0
793031,285,합천,2019-09-03 07:00,19.700000,93.0
793033,285,합천,2019-09-03 09:00,20.150000,93.0
793034,285,합천,2019-09-03 10:00,20.600000,91.0


In [18]:
## 컬럼 명 변경 및 지점 코드 drop
w1 = w1.drop(['지점'], axis=1)
w1.columns = ['loc', 'time', 'temp', 'hum']
w1.head()

,loc,time,temp,hum
0,속초,2019-01-01 01:00,-1.9,18.0
1,속초,2019-01-01 02:00,-2.2,17.0
2,속초,2019-01-01 03:00,-2.5,18.0
3,속초,2019-01-01 04:00,-3.9,20.0
4,속초,2019-01-01 05:00,-4.9,22.0


In [19]:
# 시도별 관측소 처리를 위한 사전
location = {'서울특별시':['관악산', '서울'],
            '부산광역시':['부산'],
            '대구광역시':['대구', '대구(기)'],
            '인천광역시':['강화', '백령도', '인천'],
            '광주광역시':['광주'],
            '대전광역시':['대전'],
            '울산광역시':['울산'],
            '경기도':['동두천', '수원', '양평', '이천', '파주'],
            '강원도':['강릉', '대관령', '동해', '북강릉', '북춘천', '삼척',
                   '속초', '영월', '원주', '인제', '정선군', '철원', '춘천',
                   '태백', '홍천'],
            '충청북도':['보은', '제천', '청주', '추풍령', '충주'],
            '충청남도':['금산', '보령', '부여', '서산', '천안', '홍성'],
            '전라북도': ['고창', '고창군', '군산', '남원', '부안', '순창군', '임실',
                     '장수', '전주', '정읍'],
            '전라남도':['강진군', '고흥', '광양시', '목포', '무안', '보성군',
                    '순천', '여수', '영광군', '완도', '장흥', '주암',
                    '진도(첨찰산)', '진도군', '해남', '흑산도'],
            '경상북도':['경주시', '구미', '문경', '봉화', '상주', '안동', '영덕',
                   '영주', '영천', '울릉도', '울진', '의성', '청송군', '포항'],
            '경상남도':['거제', '거창', '김해시', '남해', '밀양', '북창원', '산청',
                    '양산시', '의령군', '진주', '창원', '통영', '함양군', '합천'],
            '제주도':['고산', '서귀포', '성산', '성산포', '제주'],
            '세종특별자치시':['세종']
           }

In [20]:
def region(x):
    for key, value in location.items():
        if x in value:
            return key

In [21]:
w1['region'] = tqdm(w1['loc'].apply(region))

In [22]:
w1.isna().sum()

loc       0
time      0
temp      0
hum       0
region    0
dtype: int64

잘 처리되었습니다

In [23]:
display(w1.head())
print(w1['region'].unique())

,loc,time,temp,hum,region
0,속초,2019-01-01 01:00,-1.9,18.0,강원도
1,속초,2019-01-01 02:00,-2.2,17.0,강원도
2,속초,2019-01-01 03:00,-2.5,18.0,강원도
3,속초,2019-01-01 04:00,-3.9,20.0,강원도
4,속초,2019-01-01 05:00,-4.9,22.0,강원도


['강원도' '경기도' '인천광역시' '서울특별시' '경상북도' '충청북도' '충청남도' '대전광역시' '전라북도' '대구광역시'
 '울산광역시' '경상남도' '광주광역시' '부산광역시' '전라남도' '제주도' '세종특별자치시']


In [24]:
population = pd.read_csv("population.csv", encoding="cp949", engine="python", header=[0, 1, 2])
population

소재지(시군구)별(1)        2019                   
   소재지(시군구)별(1) 전체인구(A) (명)          도시지역 인구비율
   소재지(시군구)별(1)          소계 행정구역 인구기준(D/A*100)
0         서울특별시     9729107             100.00
1         부산광역시     3413841              99.50
2         대구광역시     2438031              99.13
3         인천광역시     2957026              97.80
4         광주광역시     1456468             100.00
5         대전광역시     1474870             100.00
6         울산광역시     1148019              95.26
7       세종특별자치시      340575              86.27
8           경기도    13239666              94.46
9           강원도     1541502              77.31
10         충청북도     1600007              79.36
11         충청남도     2123709              70.31
12         전라북도     1818917              78.82
13         전라남도     1868745              69.97
14         경상북도     2665836              74.85
15         경상남도     3362553              78.99
16      제주특별자치도      670989              94.49

## 인구 집계
- 도시지역 인구가 홈쇼핑 이용 비율이 높을 것
    - 총 인구 x 도시지역 인구비율
    - 집계, 지역별 도시지역 인구/총 도시지역 인구 -> 비율
    - 날씨 데이터에서 전국 가중합을 구할 때 가중치로 사용한다

In [25]:
population.columns = ['region', 'total', 'city']
population['mul'] = population['total']*population['city']
population.head()

,region,total,city,mul
0,서울특별시,9729107,100.00,9.729107e+08
1,부산광역시,3413841,99.50,3.396772e+08
2,대구광역시,2438031,99.13,2.416820e+08
3,인천광역시,2957026,97.80,2.891971e+08
4,광주광역시,1456468,100.00,1.456468e+08


In [26]:
pop1 = population.copy()
pop1['p'] = population['mul']/sum(population['mul'])
display(pop1)
print("sum of proportions : ", sum(pop1['p']))

,region,total,city,mul,p
0,서울특별시,9729107,100.00,9.729107e+08,0.205946
1,부산광역시,3413841,99.50,3.396772e+08,0.071903
2,대구광역시,2438031,99.13,2.416820e+08,0.051159
3,인천광역시,2957026,97.80,2.891971e+08,0.061217
4,광주광역시,1456468,100.00,1.456468e+08,0.030831
5,대전광역시,1474870,100.00,1.474870e+08,0.031220
6,울산광역시,1148019,95.26,1.093603e+08,0.023149
7,세종특별자치시,340575,86.27,2.938141e+07,0.006219
8,경기도,13239666,94.46,1.250619e+09,0.264732
9,강원도,1541502,77.31,1.191735e+08,0.025227


sum of proportions :  1.0


## Merge data
분당 데이터를 이용할 것이기 때문에 분단위로 나뉘어져 있는 형식은 우선 유지한다

In [33]:
mg = w1.groupby(['time', 'region']).mean().reset_index(drop = False)
mg.head()

,time,region,temp,hum
0,2019-01-01 01:00,강원도,-7.692857,48.714286
1,2019-01-01 01:00,경기도,-9.480000,66.000000
2,2019-01-01 01:00,경상남도,-2.635714,44.000000
3,2019-01-01 01:00,경상북도,-3.864286,42.642857
4,2019-01-01 01:00,광주광역시,-1.500000,53.000000


In [29]:
p_info = pop1[['region', 'p']] # 지역명과 가중치만 가져온다

In [37]:
df = pd.merge(mg, p_info, on = 'region')
df['TEMP'] = df['temp']*df['p']
df['HUM'] = df['hum']*df['p']
df1 = df[['time', 'region', 'TEMP', 'HUM']]
df1.head(10)

,time,region,TEMP,HUM
0,2019-01-01 01:00,강원도,-0.194065,1.228901
1,2019-01-01 02:00,강원도,-0.209202,1.264939
2,2019-01-01 03:00,강원도,-0.221815,1.346025
3,2019-01-01 04:00,강원도,-0.235870,1.358639
4,2019-01-01 05:00,강원도,-0.242717,1.419904
5,2019-01-01 06:00,강원도,-0.241636,1.468555
6,2019-01-01 07:00,강원도,-0.252808,1.535226
7,2019-01-01 08:00,강원도,-0.253528,1.583877
8,2019-01-01 09:00,강원도,-0.214067,1.535226
9,2019-01-01 10:00,강원도,-0.165235,1.380262


In [39]:
data = df1.groupby(['time']).sum().reset_index(drop=False)

In [40]:
data.head()

,time,TEMP,HUM
0,2019-01-01 01:00,-5.231287,53.536734
1,2019-01-01 02:00,-5.662894,56.822305
2,2019-01-01 03:00,-6.029719,58.833658
3,2019-01-01 04:00,-6.343332,60.112326
4,2019-01-01 05:00,-6.590452,61.873758


In [41]:
data.to_csv("지역별가중합날씨.csv", encoding='utf-8', index = False)